This version was created for comparison of different types of algoritm, when the main version is for hsowing how classification works in full dataset 

Файлы сформированные ниже содержат отчеты по testing data для обрезанного датасета 
result900_of_random_forest.txt - Random Forest
result900_of_GBC.txt - Gradient Boosting Classif
DecisionTreeClassifier - дерево решений




In [1]:
import torch
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertConfig, BertPreTrainedModel, BertForPreTraining, BertForMaskedLM
import torch.nn as nn
from tqdm import tqdm, tqdm_notebook
import os
import re
#RUBERT_PATH = 'D:\\models\\rubert_cased_L-12_H-768_A-12_pt'
#C:\Users\Lenovo\Documents\NLP\BERT\pytorch\ru_conversational_cased_L-12_H-768_A-12_pt.tar
RUBERT_PATH = 'C:\\Users\\Lenovo\\Documents\\NLP\\BERT\\pytorch\\ru_conversational_cased_L-12_H-768_A-12_pt.tar\\ru_conversational_cased_L-12_H-768_A-12_pt'
modelpath = os.path.join(RUBERT_PATH,'pytorch_model.bin')

In [2]:
tokenizer = BertTokenizer.from_pretrained(RUBERT_PATH, do_lower_case=False)
config = BertConfig.from_json_file(os.path.join(RUBERT_PATH,'bert_config.json'))
model = BertForPreTraining.from_pretrained(modelpath, config=config)
model.eval()
from torch import load
di = load(modelpath)

В связи с ограничениями вычислительной мощности, я сократила число строк до <900 (300 с каждой оценкой). И количество оценок(1 -низкий, 5 - средний, 10 - хороший). 

In [3]:
def homework4(text): 
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    segments_ids[0] = 0
    #if (len(indexed_tokens)>1):
    #    segments_ids[1] = 0
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    #if (len(indexed_tokens)>1):
    #    segments_tensors = torch.tensor([segments_ids])
    predictions = model(tokens_tensor, token_type_ids=segments_tensors)
    array_of_thensors = []
    #ln  = 0
    for i in range(len(tokenized_text)):
        #array_of_thensors = array_of_thensors + predictions[0][0][i]
        array_of_thensors.append((predictions[0][0][i].detach().numpy()))
        #ln = ln +1 
    mean = np.mean(array_of_thensors, axis = 0)
    #all information is in file "five_thousands_sorted_comments.csv" - comments with grades
    #"df_with_thensors.csv" - comment with tensor. Only 6 numbers from tensors, because when I tried to collect all 
    #length it contains more than 10 GB 
    #print(text)
    #print(mean)
    #print('######################')
    return mean

I tried to implement this algorithm with preprocessing (deleting stopords and punctuations, but accuracy was decreased by 2-3%, so i decised to skip this step. If it will be needed, I can add here proof of it. 

In [4]:
df= pd.DataFrame()
df = pd.read_csv('texts_train.txt', sep="\t", encoding='UTF-8',header=0)
df['grade'] = pd.read_csv('scores_train.txt', sep="\t", encoding='UTF-8',header=0)
df.columns = ['comment', 'grade']
print(df.head())

                                             comment  grade
0  думал, что будет лучше идея очень интересна - ...      7
1  с творчеством Головачева я познакомился посред...     10
2  то-то я и в большое неудовольствие прочитал "А...      5
3  как мне показалось местами сильно смахивает на...      6
4  от первой части книги просто оторваться не мог...      9


There is a problem, which was described via whats up chat(according the limitization of computer power of the laptop). So I reduce the number of rows and for better classification choose 500 objects of every existing classes to have 5000 rows at all

In [5]:
unique_grades = np.sort(df['grade'].unique())
print('all unique grades: ',unique_grades)
unique_grades  = [1,5,10]
print('new unique grades: ',unique_grades)

all unique grades:  [ 1  2  3  4  5  6  7  8  9 10]
new unique grades:  [1, 5, 10]


Среди уникальных оценок выбрала только 1, 5, 10 низкий, средний, высокий эмоциональный окрас соответственно. Для вычисления точности заменила их на 1,2,3 (1 = 1, 5= 2, 3=2)

In [6]:
df_new = pd.DataFrame()
comment = []
grade = []

for grades in unique_grades:
    counter = 0
    for i in range(len(df)):
    #for i in range(000/(len(unique_grades))):
        if (counter<300):
            if (grades == df['grade'][i]):
                comment.append(df['comment'][i])
                grade.append(df['grade'][i])
                counter = counter + 1 
df_new['comment'] = comment
#Заменим значения оценок на 1,2,3 
#Это облегчит подсчет ошибки 
#1 - негативный отзыв
#2 - нейтральный 
#3 - положительный
df_new['grade'] = grade
df_new['grade'] = df_new['grade'].replace(5, 2)
df_new['grade'] = df_new['grade'].replace(10, 3)
print('new unique grades: ',np.sort(df_new['grade'].unique()))
print(df_new.head())

new unique grades:  [1 2 3]
                                             comment  grade
0  Хотелось долго плеваться после того как я с тр...      1
1  ИМХО, в серии о Свароге Бушков исписался. Если...      1
2  Много был наслышан об этом авторе и вот взялся...      1
3  Да, тяжело читать безграмотную фэнтези. Хотя, ...      1
4  Попытка свести сюжет предыдущих книг в единое ...      1


In [7]:
df_new.to_csv('treeclasses_sorted_comments.csv', sep = '|') #save this dataset to csv

In [8]:
print(len(df_new))

900


In [9]:
array_of_mean_thensors = []
grade = []
comment = []
uncomtiled_rows = 0
for i in range(len(df_new)):
    try: 
        array_of_mean_thensors.append(homework4(df_new['comment'][i]))
        grade.append(df_new['grade'][i])
        comment.append(df_new['comment'][i])
    except:
        #in some cases, from my point of view, when comment is too large, the data has not processed, and it cause some errors
        uncomtiled_rows = uncomtiled_rows + 1
        #print('Too long or to short comment')

In [10]:
print(uncomtiled_rows, ' was not processed')

2  was not processed


In [20]:
df_with_thensors = pd.DataFrame()
df_with_thensors['comment'] = comment
df_with_thensors['thensor'] = array_of_mean_thensors
df_with_thensors['grade'] = grade
df_with_thensors.to_csv('df_with_thensors_900.csv', sep = '|')

In [21]:
X = array_of_mean_thensors
y = df_with_thensors['grade']

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)

In [23]:
from sklearn.metrics import accuracy_score as ac
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error as mse
def metrics(y_test, y_pred, name_of_alg): 
    mae = np.mean(abs(y_test - y_pred))
    #rmse = np.mean(abs(y_test - y_pred))
    rmse = (mse(y_test, y_pred))**0.5
    print('\bMetrics and score for ', name_of_alg,':')
    print('Accuracy score from rmse:', 100 - np.mean(100 * (rmse / y_test)))
    print('Accuracy score from mae:', 100 - np.mean(100 * (mae / y_test)))
    print('classification_report: ')
    print(classification_report(y_test, y_pred))
    print('confusion matrix')
    print(confusion_matrix(y_test, y_pred))
    print('root mse')    
    print(rmse)
    print('root mae')    
    print(mae)

In [24]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
clf = clf.fit(X_train, y_train)




In [25]:
y_pred = clf.predict(X_test)
print(metrics(y_test,y_pred, 'Random forest'))

Metrics and score for  Random forest :
Accuracy score from rmse: 46.51462002512986
Accuracy score from mae: 67.85185185185179
classification_report: 
              precision    recall  f1-score   support

           1       0.59      0.58      0.59        62
           2       0.60      0.50      0.55        64
           3       0.59      0.72      0.65        54

    accuracy                           0.59       180
   macro avg       0.59      0.60      0.59       180
weighted avg       0.60      0.59      0.59       180

confusion matrix
[[36 12 14]
 [19 32 13]
 [ 6  9 39]]
root mse
0.8595864638818418
root mae
0.5166666666666667
None


In [26]:
with open(r"result900_of_random_forest.txt", "w") as file:
    file.write('There are the result of Random forest on 5000 rows' + '\n')
    for i in range(len(y_test)):
        index = y_test.index[i]
        file.write('Comment:' + '\n')
        file.write(df_with_thensors['comment'][index] + '\n')
        file.write('y_target: ' + str(y[index]) + '\n')
        file.write('y_predicted: ' + str(y_pred[i]) + '\n')
        file.write('#######################'+ '\n')

In [27]:
from sklearn.ensemble import GradientBoostingClassifier
clf1 = GradientBoostingClassifier(n_estimators=5)

In [28]:
clf1.fit(X_train, y_train)
y_pred1 = clf1.predict(X_test)

In [29]:
print(metrics(y_test,y_pred1, 'GradientBoostingClassifier'))

Metrics and score for  GradientBoostingClassifier :
Accuracy score from rmse: 43.1991421846494
Accuracy score from mae: 65.43209876543213
classification_report: 
              precision    recall  f1-score   support

           1       0.53      0.53      0.53        62
           2       0.63      0.52      0.57        64
           3       0.59      0.72      0.65        54

    accuracy                           0.58       180
   macro avg       0.59      0.59      0.58       180
weighted avg       0.59      0.58      0.58       180

confusion matrix
[[33 14 15]
 [19 33 12]
 [10  5 39]]
root mse
0.9128709291752769
root mae
0.5555555555555556
None


In [39]:
with open(r"result900_of_GBC.txt", "w") as file:
    file.write('There are the result of Random forest on 5000 rows' + '\n')
    for i in range(len(y_test)):
        index = y_test.index[i]
        file.write('Comment:' + '\n')
        file.write(df_with_thensors['comment'][index] + '\n')
        file.write('y_target: ' + str(y[index]) + '\n')
        file.write('y_predicted: ' + str(y_pred1[i]) + '\n')
        file.write('#######################'+ '\n')

In [36]:
from sklearn.tree import DecisionTreeClassifier
clf2 = DecisionTreeClassifier()
clf2.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [37]:
pred2 = clf2.predict(X_test)

In [38]:
print(metrics(y_test,y_pred1, 'DecisionTreeClassifier'))

Metrics and score for  DecisionTreeClassifier :
Accuracy score from rmse: 43.1991421846494
Accuracy score from mae: 65.43209876543213
classification_report: 
              precision    recall  f1-score   support

           1       0.53      0.53      0.53        62
           2       0.63      0.52      0.57        64
           3       0.59      0.72      0.65        54

    accuracy                           0.58       180
   macro avg       0.59      0.59      0.58       180
weighted avg       0.59      0.58      0.58       180

confusion matrix
[[33 14 15]
 [19 33 12]
 [10  5 39]]
root mse
0.9128709291752769
root mae
0.5555555555555556
None


In [41]:
with open(r"DecisionTreeClassifier.txt", "w") as file:
    file.write('There are the result of Random forest on 5000 rows' + '\n')
    for i in range(len(y_test)):
        index = y_test.index[i]
        file.write('Comment:' + '\n')
        file.write(df_with_thensors['comment'][index] + '\n')
        file.write('y_target: ' + str(y[index]) + '\n')
        file.write('y_predicted: ' + str(pred2[i]) + '\n')
        file.write('#######################'+ '\n')